(WHAM)=
# Weighted Histogram Analysis Method

The Weighted Histogram Analysis Method (WHAM) is one of the earliest methods that take into account information from all [Intermediate States](IntermediateStates). By analyzing all the information at once, we can reduce the number of cycles and loops me must run through, improving efficiency. The precursor to WHAM and first version of multiple histogram relighting techniques came from Ferrenberg and Swendsen;{{cite|Ferrenberg1989}} WHAM was developed later for alchemical simulations{{cite|Kumar1992}}.

WHAM works on the principal that if you have a discrete number of states, you can create a histogram with discrete bins that provide you a relative probability of observing the states of interest, assuming you create the bins along whatever reaction path you have selected. In our cases, the reaction path is the alchemical variable. From these probabilities, you can calculate free energies and other observables.

## Derivation
WHAM's derivation{{cite|Kumar1992}} is best done by starting from visualizing the problem. If you are given $K$ number of states, a number of run simulations, $S$, $N_i$ samples from each simulation, and then $K$ free energies to calculate: $A_1,\,A_2,\,\cdots A_K$; we now wish to assign the reweighed potentials into bins, $B$. Terms common to [this site's definitions page](glossary_head) are preserved and while new terms are explained in context. Consider the [core free energy difference](core_eq) equation between two states $i$ and $j$ of

$\displaystyle \Delta A_{ij} = -k_B T \ln \frac{Q_j}{Q_i}$

Instead of solving this directly, we will look at the probability density, which is the ratio between the two partition functions (i.e. ignore the $-k_B T \ln$ part). Recall that the partition function of any state is often written as

$\displaystyle Q_i = \int_{V_i} \Omega_i(U) \exp\left(-\beta U \right) dU$,
$\displaystyle Q_j = \int_{V_j} \Omega_j(U) \exp\left(-\beta U \right) dU$

Where we have defined an unknown density of states for each of the thermodynamic states we are simulating, $\Omega_i$.

However, we could generalize to ''arbitrary'' integration coordinates, not just energy, which gives us:

$\displaystyle Q_i = \int_{V_i} \Omega_i(\vec{q}) \exp\left(-\beta U(\vec{q}) \right) d\vec{q}$,
$\displaystyle Q_j = \int_{V_j} \Omega_j(\vec{q}) \exp\left(-\beta U(\vec{q}) \right) d\vec{q}$

$\vec{q}$ could be coordinates, in which case $\Omega_i(\vec{q}) = 1 $, but more usually it has only one or two dimensions.

If we assume discrete states, with counts in each bin $B$, we can write the density of states out for a given simulation, $i$ as:

$\displaystyle \Omega_i(\vec{q},\lambda_i) = B_i(\vec{q},\lambda_i)\exp\left[\left(\sum_{j=0}^K \beta_i \lambda_{j,i} U(\vec{q}_j) \right)-\beta_i A_i\right]$

where the set of states available during simulation $i$ is $\left\{\lambda\right\}_i = \left\{\lambda_1, \lambda_2, \dots \lambda_K \right\}_i = \left\{\lambda_{1,i}, \lambda_{2,i}, \dots \lambda_{K,i} \right\} $ and $B_i$ is the value of the histogram bin $i$ evaluated at $\vec{q}$ and $\lambda_i$. The best estimate for the density of states is then

$\displaystyle \sum_{i=1}^S \omega_i(\vec{q})\Omega_i(\vec{q},\lambda_i)$
$\displaystyle \sum_{i=1}^S \omega_i(\vec{q})=1$

The best estimators for $\omega_i$ are the ones that minimize the statistical noise. It turns out then, that the best estimator is the one that minimize the error of $B_i$ for all $i$. The error of any individual $B_i$ is then

$\delta^2 B_i = g_i\left\langle B_i \right\rangle$

where $g_i = 1+2\tau_i$ and $\tau_i$ is the [correlation time](correlation) of a given simulation. The best estimator for the expectation of the bin value is then

$\displaystyle \hat{\left\langle B_i \right\rangle} = N_i\Omega\exp\left(\beta_i A_i - \beta\sum_{j=0}^K \lambda_{j,i} U(\vec{q}_j) \right) $.

Please note that $\Omega$ does ''not have a subscript'' in the previous equation, as it is now the density of states determined from all of the simulations.

From here you can just substitute back in to estimate $/Q_i$ to get the final WHAM equation of

$\displaystyle \hat{Q_i} = \sum_{j=1}^K \frac{\sum\limits_{x=1}^S g_x^{-1} B_x(\vec{q},\lambda_j) \exp\left[-\beta\lambda_0\vec{q}_0 - \beta\sum\limits_{l=1}^K \lambda_l U_l(\vec{q}) \right]}{\sum\limits_{y=1}^S g_y^{-1} N_y \exp\left[\beta A_y - \beta\lambda_0\vec{q}_0 - \beta\sum\limits_{m=1}^K \lambda_{m,y} U_m(\vec{q}) \right]}$

where the nought subscript indicates the conditions at the unmodified state. It is then trivial to get a free energy of this state. This is only a relative free energy though as the system must be self-consistently solved. It is common to set one of the states to a free energy of zero so you can calculate [differences in free energy](fe_facts) between states.

### Zero Width Bins
It is possible to take the bin width to zero in the WHAM formula. Although this is a limiting case, it does clean up the equation a bit and looks like

$\displaystyle \hat{A_i} = - \beta^{-1}\ln \sum_{k=1}^K \sum_{n=1}^{N_k} \frac{ \exp[-\beta U_i(\vec{q}_{kn})]}{\sum\limits_{k'=1}^K N_{k'} \, \exp[\beta A_{k'} - \beta U_{k'}(\vec{q}_{kn})]}$

which turns out to be the exact equation for [](MBAR).{{cite|Shrits2008|Shirts, M. R., and Chodera, J. D. (2008) Statistically optimal analysis of samples from multiple equilibrium states. ''J. Chem. Phys.'' 129, 129105.|http://www.citeulike.org/user/jamesr/article/3360542}}

(wham_usage)=
## Usage of WHAM
Given a particular implementation of WHAM ([see below](#dl_wham)), one may be tempted just to let the analysis give you a black-box result. You should always understand what is happening and so the results should not be taken on blind faith alone.

Since WHAM collects data from all states, you will need to calculate $\Delta U_{k,j}(\vec{q})$ for all $ j=1,2,\cdots,K$ states. Even though $\Delta U_{k,k}(\vec{q})$ does not ''need'' to be calculated since it ''should'' be zero, it is highly recommended that you do. This lets you check to see if the re-evaluation function is working as intended, and help you identify possible errors. Although this does not tell you what the error is, it will at least tell you there is one. One possible source of error is the output of your coordinate files should be higher accuracy than a standard PDB files, $10^5 $&Aring; may be sufficient but binary format coordinates are preferred.

Because you are applying a discrete set and finite number of bins, there is a bias introduced since all variables must fall into the bins. This is the predominate problem with WHAM and everything that comes with it, so exercise caution. Error estimates are not directly available for WHAM, and so methods such as [bootstrap sampling](bootstrap) are required.

(dl_wham)=
## Downloading WHAM
Because WHAM requires solving sets of non-linear equations, it is not recommended for beginners to write their own. Fortunately, many simulation packages, such as GROMACS and CHARMM already include WHAM driven free energy calculations.{{cite|Hub2010}}{{cite|Souaille2001}}{{cite|Wang2006}} There are also [also other standalone packages available](http://membrane.urmc.rochester.edu/content/wham) and so there should not be a need to write your own.

## References
<references>
{{cite|Ferrenberg1989|Ferrenberg, A. M., and Swendsen, R. H. (1989) Optimized Monte Carlo Data Analysis. ''Phys. Rev. Lett'' 63, 1195–1198.|http://www.citeulike.org/user/dgront/article/774372}}

{{cite|Kumar1992|Kumar, S., Bouzida, D., Swendsen, R. H., Kollman, P. A., and Rosenberg, J. M. (1992) The weighted histogram analysis method for free-energy calculations on biomolecules. I. The method. ''J. Comput. Chem.'' 13, 1011–1021.|http://www.citeulike.org/user/dgront/article/774373}}

{{cite|Hub2010|Hub, J. S., de Groot, B. L., and van der Spoel, D. (2010) g_wham–A Free Weighted Histogram Analysis Implementation Including Robust Error and Autocorrelation Estimates. ''J. Chem. Theo. Comput.'' 6, 3713–3720.|http://www.citeulike.org/user/agrossfield/article/8443854}}

{{cite|Souaille2001|Souaille, M., and Roux, B. (2001) Extension to the weighted histogram analysis method: combining umbrella sampling with free energy calculations. ''Comput. Phys. Commun.'' 135, 40–57.|http://www.citeulike.org/user/girayenkavi/article/2910539}}

{{cite|Wang2006|Wang, J., Deng, Y., and Roux, B. (2006) Absolute Binding Free Energy Calculations Using Molecular Dynamics Simulations with Restraining Potentials. ''Biophys. J.'' 91, 2798–2814.|http://www.citeulike.org/user/kupopo/article/867499}}
</references>